In [1]:
import pandas as pd
from pathlib import Path

In [2]:
video_tokens_per_sec_nominal = 263
audio_tokens_per_sec_nominal = 32

In [3]:
pricing = pd.DataFrame([
    {
        'model': 'gemini-2.5-pro-preview-05-06',
        'input_non_audio': 2.50,
        'input_audio': 2.50,
        'output_non_thinking': 15.00,
        'output_thinking': 15.00,
    },
    {
        'model': 'gemini-2.5-flash-preview-05-20',
        'input_non_audio': 0.15,
        'input_audio': 1.00,
        'output_non_thinking': 0.60,
        'output_thinking': 3.50,
    },
    {
        'model': 'gemini-2.0-flash',
        'input_non_audio': 0.10,
        'input_audio': 0.70,
        'output_non_thinking': 0.40,
        'output_thinking': 0.40,
    },
    {
        'model': 'gemini-2.0-flash-lite',
        'input_non_audio': 0.075,
        'input_audio': 0.075,
        'output_non_thinking': 0.30,
        'output_thinking': 0.30,
    },
]).set_index('model')

In [4]:
pricing

,input_non_audio,input_audio,output_non_thinking,output_thinking
model,,,,
gemini-2.5-pro-preview-05-06,2.500,2.500,15.0,15.0
gemini-2.5-flash-preview-05-20,0.150,1.000,0.6,3.5
gemini-2.0-flash,0.100,0.700,0.4,0.4
gemini-2.0-flash-lite,0.075,0.075,0.3,0.3


In [5]:
metadata_table = pd.read_pickle('./output/gemini_analysis_metadata_20250526_124112.pkl')

In [6]:
metadata_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   video                            16 non-null     object 
 1   prompt                           16 non-null     object 
 2   model                            16 non-null     object 
 3   video_local_path                 16 non-null     object 
 4   video_remote_path                16 non-null     object 
 5   video_duration_minutes           16 non-null     float64
 6   video_duration_minutes_returned  16 non-null     float64
 7   video_size_mb                    16 non-null     float64
 8   video_size_mb_returned           16 non-null     float64
 9   prompt_tokens_precalculated      16 non-null     int64  
 10  video_tokens_precalculated       16 non-null     int64  
 11  prompt_token_count               16 non-null     int64  
 12  thoughts_token_count    

In [7]:
estimates = (
    metadata_table
    .set_index([
        'video',
        'prompt',
        'model'
    ])
    .assign(thoughts_token_count=lambda x: x['thoughts_token_count'].fillna(0))
)

In [8]:
(
    estimates['total_token_count'] - 
    (
        estimates['prompt_token_count'] +
        estimates['thoughts_token_count'] +
        estimates['candidates_token_count']
    )
)

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06      0.0
                     gemini-2.5-flash-preview-05-20    0.0
                     gemini-2.0-flash                  0.0
                     gemini-2.0-flash-lite             0.0
          tq_prompt  gemini-2.5-pro-preview-05-06      0.0
                     gemini-2.5-flash-preview-05-20    0.0
                     gemini-2.0-flash                  0.0
                     gemini-2.0-flash-lite             0.0
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06      0.0
                     gemini-2.5-flash-preview-05-20    0.0
                     gemini-2.0-flash                  0.0
                     gemini-2.0-flash-lite             0.0
          tq_prompt  gemini-2.5-pro-preview-05-06      0.0
                     gemini-2.5-flash-preview-05-20    0.0
                     gemini-2.0-flash                  0.0
                     gemini-2.0-flash-lite             0.0
dtyp

In [9]:
(
    estimates['prompt_token_count'] - 
    (
        estimates['prompt_tokens_detail_text'] +
        estimates['prompt_tokens_detail_video'] +
        estimates['prompt_tokens_detail_audio']
    )
)

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  0
                     gemini-2.0-flash-lite             0
          tq_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  0
                     gemini-2.0-flash-lite             0
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  0
                     gemini-2.0-flash-lite             0
          tq_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  0
                     gemini-2.0-flash-lite             0
dtype: int64

In [10]:
estimates['prompt_tokens_precalculated'] - estimates['prompt_tokens_detail_text']

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  1
                     gemini-2.0-flash-lite             1
          tq_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  1
                     gemini-2.0-flash-lite             1
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  1
                     gemini-2.0-flash-lite             1
          tq_prompt  gemini-2.5-pro-preview-05-06      0
                     gemini-2.5-flash-preview-05-20    0
                     gemini-2.0-flash                  1
                     gemini-2.0-flash-lite             1
dtype: int64

In [11]:
(
    estimates['video_tokens_precalculated'] - 
    (
        estimates['prompt_tokens_detail_video'] +
        estimates['prompt_tokens_detail_audio']
    )
)

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06         1
                     gemini-2.5-flash-preview-05-20       1
                     gemini-2.0-flash                  2626
                     gemini-2.0-flash-lite             2626
          tq_prompt  gemini-2.5-pro-preview-05-06         1
                     gemini-2.5-flash-preview-05-20       1
                     gemini-2.0-flash                  2626
                     gemini-2.0-flash-lite             2626
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06         1
                     gemini-2.5-flash-preview-05-20       1
                     gemini-2.0-flash                  1686
                     gemini-2.0-flash-lite             1686
          tq_prompt  gemini-2.5-pro-preview-05-06         1
                     gemini-2.5-flash-preview-05-20       1
                     gemini-2.0-flash                  1686
                     gemini-2.0-flash-lite      

In [12]:
estimates['prompt_tokens_detail_video'].div(estimates['video_duration_minutes_returned']*60)

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06      263.000000
                     gemini-2.5-flash-preview-05-20    263.000000
                     gemini-2.0-flash                  258.888889
                     gemini-2.0-flash-lite             258.888889
          tq_prompt  gemini-2.5-pro-preview-05-06      263.000000
                     gemini-2.5-flash-preview-05-20    263.000000
                     gemini-2.0-flash                  258.888889
                     gemini-2.0-flash-lite             258.888889
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06      263.000000
                     gemini-2.5-flash-preview-05-20    263.000000
                     gemini-2.0-flash                  258.142857
                     gemini-2.0-flash-lite             258.142857
          tq_prompt  gemini-2.5-pro-preview-05-06      263.000000
                     gemini-2.5-flash-preview-05-20    263.000000
                     gem

In [13]:
video_tokens_per_sec_nominal

263

In [14]:
estimates['prompt_tokens_detail_audio'].div(estimates['video_duration_minutes_returned']*60)

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06      32.000000
                     gemini-2.5-flash-preview-05-20    32.000000
                     gemini-2.0-flash                  24.893162
                     gemini-2.0-flash-lite             24.893162
          tq_prompt  gemini-2.5-pro-preview-05-06      32.000000
                     gemini-2.5-flash-preview-05-20    32.000000
                     gemini-2.0-flash                  24.893162
                     gemini-2.0-flash-lite             24.893162
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06      32.000000
                     gemini-2.5-flash-preview-05-20    32.000000
                     gemini-2.0-flash                  24.821429
                     gemini-2.0-flash-lite             24.821429
          tq_prompt  gemini-2.5-pro-preview-05-06      32.000000
                     gemini-2.5-flash-preview-05-20    32.000000
                     gemini-2.0-flash 

In [15]:
audio_tokens_per_sec_nominal

32

In [16]:
estimates['thoughts_token_count'].div(estimates['video_duration_minutes_returned'])

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06       925.641026
                     gemini-2.5-flash-preview-05-20     681.282051
                     gemini-2.0-flash                     0.000000
                     gemini-2.0-flash-lite                0.000000
          tq_prompt  gemini-2.5-pro-preview-05-06       708.461538
                     gemini-2.5-flash-preview-05-20     644.615385
                     gemini-2.0-flash                     0.000000
                     gemini-2.0-flash-lite                0.000000
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06       996.428571
                     gemini-2.5-flash-preview-05-20    1962.000000
                     gemini-2.0-flash                     0.000000
                     gemini-2.0-flash-lite                0.000000
          tq_prompt  gemini-2.5-pro-preview-05-06       841.714286
                     gemini-2.5-flash-preview-05-20     549.428571
          

In [17]:
estimates['candidates_token_count'].div(estimates['video_duration_minutes_returned'])

video     prompt     model                         
lh_video  lh_prompt  gemini-2.5-pro-preview-05-06      661.282051
                     gemini-2.5-flash-preview-05-20    535.897436
                     gemini-2.0-flash                  222.564103
                     gemini-2.0-flash-lite             174.871795
          tq_prompt  gemini-2.5-pro-preview-05-06      412.564103
                     gemini-2.5-flash-preview-05-20    357.179487
                     gemini-2.0-flash                  213.589744
                     gemini-2.0-flash-lite             111.538462
tq_video  lh_prompt  gemini-2.5-pro-preview-05-06      788.571429
                     gemini-2.5-flash-preview-05-20    562.285714
                     gemini-2.0-flash                  418.285714
                     gemini-2.0-flash-lite             361.285714
          tq_prompt  gemini-2.5-pro-preview-05-06      718.714286
                     gemini-2.5-flash-preview-05-20    454.285714
                     gem

In [22]:
estimates = (
    estimates
    .join(
        pricing,
        how='left',
        on='model'
    )
    .assign(
        input_text_costs=lambda x: x['input_non_audio'] * x['prompt_tokens_detail_text'] / 1000000,
        input_video_costs=lambda x: x['input_non_audio'] * x['prompt_tokens_detail_video'] / 1000000,
        input_audio_costs=lambda x: x['input_audio'] * x['prompt_tokens_detail_audio'] / 1000000,
        output_non_thinking_costs=lambda x: x['output_non_thinking'] * x['candidates_token_count'] / 1000000,
        output_thinking_costs=lambda x: x['output_thinking'] * x['thoughts_token_count'] / 1000000,
        total_costs=lambda x: (
            x['input_text_costs'] +
            x['input_video_costs'] +
            x['input_audio_costs'] +
            x['output_non_thinking_costs'] +
            x['output_thinking_costs']
        ),
        input_video_costs_per_minute=lambda x: x['input_video_costs']/x['video_duration_minutes_returned'],
        input_audio_costs_per_minute=lambda x: x['input_audio_costs']/x['video_duration_minutes_returned'],
        output_non_thinking_costs_per_minute=lambda x: x['output_non_thinking_costs']/x['video_duration_minutes_returned'],
        output_thinking_costs_per_minute=lambda x: x['output_thinking_costs']/x['video_duration_minutes_returned'],
        total_costs_per_minute =lambda x: (
            x['input_video_costs_per_minute'] +
            x['input_audio_costs_per_minute'] +
            x['output_non_thinking_costs_per_minute'] +
            x['output_thinking_costs_per_minute']
        )
    )
)

In [23]:
estimates

video_local_path  \
video    prompt    model                                                                               
lh_video lh_prompt gemini-2.5-pro-preview-05-06    ../videos/Zearn Screen Recording 2025-04-25 at...   
                   gemini-2.5-flash-preview-05-20  ../videos/Zearn Screen Recording 2025-04-25 at...   
                   gemini-2.0-flash                ../videos/Zearn Screen Recording 2025-04-25 at...   
                   gemini-2.0-flash-lite           ../videos/Zearn Screen Recording 2025-04-25 at...   
         tq_prompt gemini-2.5-pro-preview-05-06    ../videos/Zearn Screen Recording 2025-04-25 at...   
                   gemini-2.5-flash-preview-05-20  ../videos/Zearn Screen Recording 2025-04-25 at...   
                   gemini-2.0-flash                ../videos/Zearn Screen Recording 2025-04-25 at...   
                   gemini-2.0-flash-lite           ../videos/Zearn Screen Recording 2025-04-25 at...   
tq_video lh_prompt gemini-2.5-pro-preview-05-06           ../videos/zearn_g8m2l4_brief_confusion.mp4   
                   gemini-2.5-flash-preview-05-20         ../videos/zearn_g8m2l4_brief_confusion.mp4   
                   gemini-2.0-flash                       ../videos/zearn_g8m2l4_brief_confusion.mp4   
                   gemini-2.0-flash-lite                  ../videos/zearn_g8m2l4_brief_confusion.mp4   
         tq_prompt gemini-2.5-pro-preview-05-06           ../videos/zearn_g8m2l4_brief_confusion.mp4   
                   gemini-2.5-flash-preview-05-20         ../videos/zearn_g8m2l4_brief_confusion.mp4   
                   gemini-2.0-flash                       ../videos/zearn_g8m2l4_brief_confusion.mp4   
                   gemini-2.0-flash-lite                  ../videos/zearn_g8m2l4_brief_confusion.mp4   

                                                    video_remote_path  \
video    prompt    model                                                
lh_video lh_prompt gemini-2.5-pro-preview-05-06    files/9i70vkwtb5fp   
                   gemini-2.5-flash-preview-05-20  files/9i70vkwtb5fp   
                   gemini-2.0-flash                files/9i70vkwtb5fp   
                   gemini-2.0-flash-lite           files/9i70vkwtb5fp   
         tq_prompt gemini-2.5-pro-preview-05-06    files/9i70vkwtb5fp   
                   gemini-2.5-flash-preview-05-20  files/9i70vkwtb5fp   
                   gemini-2.0-flash                files/9i70vkwtb5fp   
                   gemini-2.0-flash-lite           files/9i70vkwtb5fp   
tq_video lh_prompt gemini-2.5-pro-preview-05-06    files/fnpocesuq6rx   
                   gemini-2.5-flash-preview-05-20  files/fnpocesuq6rx   
                   gemini-2.0-flash                files/fnpocesuq6rx   
                   gemini-2.0-flash-lite           files/fnpocesuq6rx   
         tq_prompt gemini-2.5-pro-preview-05-06    files/fnpocesuq6rx   
                   gemini-2.5-flash-preview-05-20  files/fnpocesuq6rx   
                   gemini-2.0-flash                files/fnpocesuq6rx   
                   gemini-2.0-flash-lite           files/fnpocesuq6rx   

                                                   video_duration_minutes  \
video    prompt    model                                                    
lh_video lh_prompt gemini-2.5-pro-preview-05-06                  3.883333   
                   gemini-2.5-flash-preview-05-20                3.883333   
                   gemini-2.0-flash                              3.883333   
                   gemini-2.0-flash-lite                         3.883333   
         tq_prompt gemini-2.5-pro-preview-05-06                  3.883333   
                   gemini-2.5-flash-preview-05-20                3.883333   
                   gemini-2.0-flash                              3.883333   
                   gemini-2.0-flash-lite                         3.883333   
tq_video lh_prompt gemini-2.5-pro-preview-05-06                  2.316667   
                   gemini-2.5-flash-preview-05-20